In [13]:
import numpy as np
import netCDF4 as nc
from os import path

def add_attribution(ncfin_coastal, ncfin_runoff, ncfin_compound, ncfout, toexclude, threshold=0.05):
    with nc.Dataset(ncfin_coastal) as src_coastal, \
         nc.Dataset(ncfin_runoff) as src_runoff, \
         nc.Dataset(ncfin_compound) as src_compound, \
         nc.Dataset(ncfout, "w") as dst:
        # copy global attributes all at once via dictionary
        src1 = src_compound
        dst.setncatts(src1.__dict__)
        # copy dimensions
        for name, dimension in src1.dimensions.items():
            dst.createDimension(
                name, (len(dimension) if not dimension.isunlimited() else None))
        # copy all file data except for the excluded
        for name, variable in src1.variables.items():
            if name not in toexclude:
                x = dst.createVariable(name, variable.datatype, variable.dimensions)
                # copy variable attributes all at once via dictionary
                dst[name].setncatts(src1[name].__dict__)
                dst[name][:] = src1[name][:]
            if name == 'zeta_max':
                variable_zeta_max = variable
                
        # add values

        def add_variable(src, dst, dst_name, long_name, standard_name):
            x = dst.createVariable(dst_name, variable_zeta_max.datatype, variable_zeta_max.dimensions, fill_value=variable_zeta_max._FillValue)
            attr = src['zeta_max'].__dict__
            attr['long_name'] = long_name
            attr['standard_name'] = standard_name
            dst[dst_name].setncatts(attr)
        
        src1 = src_compound
        add_variable(src1, dst, 'zeta_max_coastal', 'maximum water surface elevation for coastal', 'zeta_max_coastal')
        add_variable(src1, dst, 'zeta_max_runoff', 'maximum water surface elevation for runoff', 'zeta_max_runoff')
        add_variable(src1, dst, 'zeta_max_compound', 'maximum water surface elevation for compound', 'zeta_max_compound')
        add_variable(src1, dst, 'zeta_max_individual', 'maximum water surface elevation for coastal and runoff', 'zeta_max_individual')
        add_variable(src1, dst, 'zeta_max_compound_minus_individual', 'maximum water surface elevation for compound minus individual', 'zeta_max_compound_minus_individual')
        add_variable(src1, dst, 'zeta_max_attribution', 'maximum water surface elevation attribution - 1: coastal, 2: coastal, compound, 3: runoff compound, 4: runoff', 'zeta_max_attribution')
        
        zeta_max_coastal = src_coastal['zeta_max'][:]
        zeta_max_runoff = src_runoff['zeta_max'][:]
        zeta_max_compound = src_compound['zeta_max'][:]

        zeta_max_coastal[np.where(zeta_max_coastal.mask)] = np.nan
        zeta_max_runoff[np.where(zeta_max_runoff.mask)] = np.nan
        zeta_max_compound[np.where(zeta_max_compound.mask)] = np.nan
        
        zeta_max_coastal[zeta_max_coastal < -99998.0] = np.nan
        zeta_max_runoff[zeta_max_runoff < -99998.0] = np.nan
        zeta_max_compound[zeta_max_compound < -99998.0] = np.nan

        zeta_max_individual = np.nanmax([zeta_max_coastal, zeta_max_runoff], axis=0)
        zeta_max_compound_minus_individual = zeta_max_compound - zeta_max_individual
        
        # zeta_max_individual[zeta_max_individual < -99998.0] = np.nan
        # zeta_max_compound_minus_individual[zeta_max_compound_minus_individual < -99998.0] = np.nan

        nn = len(zeta_max_coastal)
        iscoastal = np.array([1 if zeta_max_coastal[i] >= zeta_max_runoff[i] or (not np.isnan(zeta_max_coastal[i]) and np.isnan(zeta_max_runoff[i])) else 0 for i in range(nn)]).astype(np.float64)
        iscompound = np.array([1 if zeta_max_compound[i] - threshold >= max(zeta_max_coastal[i], zeta_max_runoff[i]) else 0 for i in range(nn)]).astype(np.float64)
        zeta_max_attribution = np.array([1.0 if iscoa == 1 and iscom == 0 else 2.0 if iscoa == 1 and iscom == 1 else 3.0 if iscoa == 0 and iscom == 1 else 4.0 for iscoa, iscom in zip(iscoastal, iscompound)])
        iii = 195379
        print('1 zeta_max_attribution[195379]',zeta_max_attribution[iii], iscoastal[iii], iscompound[iii], not np.isnan(zeta_max_coastal[iii]), np.isnan(zeta_max_runoff[iii]), zeta_max_runoff[iii] == zeta_max_runoff[iii], not np.isnan(zeta_max_coastal[iii]) and np.isnan(zeta_max_runoff[iii]))
        zeta_max_attribution[np.all([np.isnan(zeta_max_coastal), iscoastal == 1.0], axis=0)] = -99999.0
        print('2 zeta_max_attribution[195379]',zeta_max_attribution[iii])
        zeta_max_attribution[np.all([np.isnan(zeta_max_runoff), iscoastal == 0.0], axis=0)] = -99999.0
        print('3 zeta_max_attribution[195379]',zeta_max_attribution[iii])
        
        dst['zeta_max_coastal'][:] = zeta_max_coastal
        dst['zeta_max_runoff'][:] = zeta_max_runoff
        dst['zeta_max_compound'][:] = zeta_max_compound
        dst['zeta_max_individual'][:] = zeta_max_individual
        dst['zeta_max_compound_minus_individual'][:] = zeta_max_compound_minus_individual
        dst['zeta_max_attribution'][:] = zeta_max_attribution


basedir = "/projects/storm_surge/COMT/sbunya/runs/EmbedVEW1DtoLargeModel/NC/Florence"

ncfin_coastal = "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_noflows_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_swan2noswan_ERA52OWI2ERA5/maxele.63.nc"
ncfin_runoff = "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_nomet/maxele.63.nc"
ncfin_compound = "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele.63.nc"
ncfou = "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele_attribution_rev1.63.nc"

# toexclude = ['depth', 'zeta_max', 'time_of_zeta_max']
toexclude = ['zeta_max', 'time_of_zeta_max']

add_attribution(path.join(basedir, ncfin_coastal), \
                path.join(basedir, ncfin_runoff), \
                path.join(basedir, ncfin_compound), \
                path.join(basedir, ncfou), \
                toexclude)


/tmp/ipykernel_29038/1082393154.py:56: RuntimeWarning: All-NaN axis encountered
  zeta_max_individual = np.nanmax([zeta_max_coastal, zeta_max_runoff], axis=0)


1 zeta_max_attribution[195379] 1.0 1.0 0.0 True True False True
2 zeta_max_attribution[195379] 1.0
3 zeta_max_attribution[195379] 1.0
